In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Othercomputers/My\ Laptop/291-deepgen/Lyrics2Melody/Lyrics-Conditioned-Neural-Melody-Generation
!pip install py-midi
!pip install pretty_midi

/content/drive/Othercomputers/My Laptop/291-deepgen/Lyrics2Melody/Lyrics-Conditioned-Neural-Melody-Generation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 5.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 6.3 MB/s 
     |████████████████████████████████| 51 kB 8.8 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=abed38efa2e8e3cdc66fd67882328c6bbc9ab914878c1bee77d28b81666cfb4e
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [ ]:

# This notebook is used to generate triplet of music attributes (melody) for lyrics in the test data.


In [1]:

import numpy as np
import tensorflow as tf
import midi_statistics
import utils
import pickle
import matplotlib.pyplot as plt


In [2]:
def do_generate_songs(model_path, data):
  generated_songs = []

  model_stats = {}
  model_stats['stats_scale_tot'] = 0
  model_stats['stats_repetitions_2_tot'] = 0
  model_stats['stats_repetitions_3_tot'] = 0
  model_stats['stats_span_tot'] = 0
  model_stats['stats_unique_tones_tot'] = 0
  model_stats['stats_avg_rest_tot'] = 0
  model_stats['num_of_null_rest_tot'] = 0
  model_stats['best_scale_score'] = 0
  model_stats['best_repetitions_2'] = 0
  model_stats['best_repetitions_3'] = 0
  model_stats['num_perfect_scale'] = 0
  model_stats['num_good_songs'] = 0
  model_stats['avg_songlength'] = 0

  with tf.compat.v1.Session(graph=tf.Graph()) as sess:
      tf.compat.v1.saved_model.loader.load(sess, [], model_path)
      graph = tf.compat.v1.get_default_graph()
      keep_prob = graph.get_tensor_by_name("model/keep_prob:0")
      input_metadata = graph.get_tensor_by_name("model/input_metadata:0")
      input_songdata = graph.get_tensor_by_name("model/input_data:0")
      output_midi = graph.get_tensor_by_name("output_midi:0")
      feed_dict = {}
      feed_dict[keep_prob.name] = 1
      condition = []
      feed_dict[input_metadata.name] = condition


      for i in range(len(data)):
          feed_dict[input_songdata.name] = np.random.uniform(size=(1, 20, 3))
          condition.append(np.split(data[i][3 * 20:], 20))
          feed_dict[input_metadata.name] = condition
          generated_features = sess.run(output_midi, feed_dict)
          sample = [x[0, :] for x in generated_features]
          sample = midi_statistics.tune_song(utils.discretize(sample))
          generated_songs.append(sample)
          condition = []

          # get stats
          stats = midi_statistics.get_all_stats(sample)
          model_stats['stats_scale_tot'] += stats['scale_score']
          model_stats['stats_repetitions_2_tot'] += float(stats['repetitions_2'])
          model_stats['stats_repetitions_3_tot'] += float(stats['repetitions_3'])
          model_stats['stats_unique_tones_tot'] += float(stats['tones_unique'])
          model_stats['stats_span_tot'] += stats['tone_span']
          model_stats['stats_avg_rest_tot'] += stats['average_rest']
          model_stats['num_of_null_rest_tot'] += stats['num_null_rest']
          model_stats['best_scale_score'] = max(stats['scale_score'], model_stats['best_scale_score'])
          model_stats['best_repetitions_2'] = max(stats['repetitions_2'], model_stats['best_repetitions_2'])
          model_stats['best_repetitions_3'] = max(stats['repetitions_3'], model_stats['best_repetitions_3'])
          model_stats['avg_songlength'] += stats['songlength']

          if stats['scale_score'] == 1.0 and stats['tones_unique'] > 3 \
              and stats['tone_span'] > 4 and stats['num_null_rest'] > 8 and stats['tone_span'] < 13\
              and stats['repetitions_2'] > 4:
              model_stats['num_good_songs'] += 1
      
      model_stats['avg_songlength'] /= len(data)

  return generated_songs, model_stats

In [28]:
train_data = np.load('./data/processed_dataset_matrices/train_data_matrix.npy')
valid_data = np.load('./data/processed_dataset_matrices/valid_data_matrix.npy')
test_data = np.load('./data/processed_dataset_matrices/test_data_matrix.npy')
train_data.shape, valid_data.shape, test_data.shape

((11149, 460), (1051, 460), (1051, 460))

In [29]:
d_midi_train = train_data[:,:60]
d_midi_valid = valid_data[:,:60]
d_midi_test = test_data[:,:60]

In [53]:
from sklearn.preprocessing import OneHotEncoder

In [190]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(d_midi_train)

OneHotEncoder(handle_unknown='ignore')

In [191]:
a = enc.transform(d_midi_train).toarray()

In [199]:
b = enc.transform(d_midi_valid).toarray()
b_inv = enc.inverse_transform(b)

In [203]:
b_inv[13:14]

array([[39.0, 2.0, 0.0, 36.0, 1.0, 0.0, 32.0, 4.0, 0.0, 37.0, 2.0, 0.0,
        None, 1.0, 0.0, 37.0, 4.0, 0.0, 39.0, 2.0, 0.0, 27.0, 1.0, 0.0,
        39.0, 4.0, 0.0, 32.0, 2.0, 0.0, 32.0, 1.0, 0.0, 32.0, 2.0, 0.0,
        32.0, 0.5, 0.0, 42.0, 4.0, 1.0, 41.0, 4.0, 0.0, 30.0, 8.0, 0.0,
        32.0, 1.0, 0.0, 32.0, 6.0, 0.0, 29.0, 1.0, 0.0, 41.0, 2.0, 0.0]],
      dtype=object)

In [210]:
imputer.transform(b_inv[13:14])

array([[39.0, 2.0, 0.0, 36.0, 1.0, 0.0, 32.0, 4.0, 0.0, 37.0, 2.0, 0.0,
        None, 1.0, 0.0, 37.0, 4.0, 0.0, 39.0, 2.0, 0.0, 27.0, 1.0, 0.0,
        39.0, 4.0, 0.0, 32.0, 2.0, 0.0, 32.0, 1.0, 0.0, 32.0, 2.0, 0.0,
        32.0, 0.5, 0.0, 42.0, 4.0, 1.0, 41.0, 4.0, 0.0, 30.0, 8.0, 0.0,
        32.0, 1.0, 0.0, 32.0, 6.0, 0.0, 29.0, 1.0, 0.0, 41.0, 2.0, 0.0]],
      dtype=object)

In [220]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

imputer.fit(d_midi_train)
b_inv = np.array(b_inv, dtype=np.float64)
b_inv = imputer.transform(b_inv)
b_inv[13:14]

C:\Users\EJ\anaconda3\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


array([[39. ,  2. ,  0. , 36. ,  1. ,  0. , 32. ,  4. ,  0. , 37. ,  2. ,
         0. , 69. ,  1. ,  0. , 37. ,  4. ,  0. , 39. ,  2. ,  0. , 27. ,
         1. ,  0. , 39. ,  4. ,  0. , 32. ,  2. ,  0. , 32. ,  1. ,  0. ,
        32. ,  2. ,  0. , 32. ,  0.5,  0. , 42. ,  4. ,  1. , 41. ,  4. ,
         0. , 30. ,  8. ,  0. , 32. ,  1. ,  0. , 32. ,  6. ,  0. , 29. ,
         1. ,  0. , 41. ,  2. ,  0. ]])

C:\Users\EJ\AppData\Local\Temp\ipykernel_28264\1637327877.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.array(b_inv, dtype=np.float)[13]


array([39. ,  2. ,  0. , 36. ,  1. ,  0. , 32. ,  4. ,  0. , 37. ,  2. ,
        0. ,  nan,  1. ,  0. , 37. ,  4. ,  0. , 39. ,  2. ,  0. , 27. ,
        1. ,  0. , 39. ,  4. ,  0. , 32. ,  2. ,  0. , 32. ,  1. ,  0. ,
       32. ,  2. ,  0. , 32. ,  0.5,  0. , 42. ,  4. ,  1. , 41. ,  4. ,
        0. , 30. ,  8. ,  0. , 32. ,  1. ,  0. , 32. ,  6. ,  0. , 29. ,
        1. ,  0. , 41. ,  2. ,  0. ])

In [168]:
imputer.fit(a)
b = enc.transform(d_midi_valid).toarray()
rr = imputer.transform(b[0])
b[b!=rr]

C:\Users\EJ\anaconda3\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [128]:
np.expand_dims(a[0].reshape(636, 3).transpose(1,0), axis=1).shape

(3, 1, 636)

In [124]:
np.expand_dims(d_midi_train.reshape(d_midi_train.shape[0], 20, 3).transpose(0,2,1),axis=2).shape

(11149, 3, 1, 20)

In [114]:
d_midi_train[0].shape, a[0].shape

((60,), (1908,))

In [87]:
aa = np.expand_dims(np.array(np.split(d_midi_train[0], 20)).transpose(1,0),axis=1)

In [88]:
aa

array([[[76.  , 76.  , 76.  , 76.  , 72.  , 77.  , 76.  , 72.  , 76.  ,
         74.  , 74.  , 72.  , 76.  , 74.  , 74.  , 72.  , 76.  , 74.  ,
         74.  , 72.  ]],

       [[ 0.25,  0.75,  0.25,  3.  ,  1.  ,  2.  ,  1.  ,  3.  ,  1.  ,
          0.5 ,  1.  ,  4.  ,  1.  ,  1.  ,  1.  ,  4.  ,  2.  ,  0.75,
          1.  ,  6.  ]],

       [[ 0.  ,  0.  ,  0.  ,  1.  ,  8.  ,  0.  ,  0.  ,  0.  ,  8.  ,
          0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,
          0.  ,  0.  ]]])

In [105]:
d_midi_train_new = np.expand_dims(d_midi_train.reshape(d_midi_train.shape[0],20,3).transpose(0,2,1),axis=2)

In [106]:
d_midi_train_new.shape

(11149, 3, 1, 20)

In [80]:
b = enc.transform(d_midi_valid).toarray()
b.shape

(1051, 1908)

In [82]:
b_r = enc.inverse_transform(b)

In [83]:
b_r.shape

(1051, 60)

In [30]:
def get_midi_note_rest(d_midi):
    d = d_midi.reshape(d_midi.shape[0], 20, 3).T
    return d[0][0], d[1][0], d[2][0]

In [33]:
midi_num_tr, note_dur_tr, rest_dur_tr = get_midi_note_rest(d_midi_train)
midi_num_va, note_dur_va, rest_dur_va = get_midi_note_rest(d_midi_valid)
midi_num_te, note_dur_te, rest_dur_te = get_midi_note_rest(d_midi_test)

In [47]:
np.unique(np.concatenate((midi_num_tr, midi_num_va, midi_num_te)))

array([ 12.,  16.,  19.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,
        30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
        41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,
        52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,
        63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,
        74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,
        85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  95.,  96.,
        97.,  98., 119.])

In [49]:
np.unique(np.concatenate((note_dur_tr, note_dur_va, note_dur_te)))

array([ 0.25,  0.5 ,  0.75,  1.  ,  1.5 ,  2.  ,  3.  ,  4.  ,  6.  ,
        8.  , 16.  , 32.  ])

In [50]:
np.unique(np.concatenate((rest_dur_tr, rest_dur_va, rest_dur_te)))

array([ 0.,  1.,  2.,  4.,  8., 16., 32.])

In [51]:
MIDI_NUM_CAND = [ 12.,  16.,  19.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,
        30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
        41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,
        52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,
        63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,
        74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,
        85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  95.,  96.,
        97.,  98., 119.]
DUR_CAND = [ 0.25,  0.5 ,  0.75,  1.  ,  1.5 ,  2.  ,  3.  ,  4.  ,  6.  ,
        8.  , 16.  , 32. ]
REST_CAND = [ 0.,  1.,  2.,  4.,  8., 16., 32.]

In [52]:
d_midi_train[0]

array([76.  ,  0.25,  0.  , 76.  ,  0.75,  0.  , 76.  ,  0.25,  0.  ,
       76.  ,  3.  ,  1.  , 72.  ,  1.  ,  8.  , 77.  ,  2.  ,  0.  ,
       76.  ,  1.  ,  0.  , 72.  ,  3.  ,  0.  , 76.  ,  1.  ,  8.  ,
       74.  ,  0.5 ,  0.  , 74.  ,  1.  ,  0.  , 72.  ,  4.  ,  0.  ,
       76.  ,  1.  ,  1.  , 74.  ,  1.  ,  0.  , 74.  ,  1.  ,  0.  ,
       72.  ,  4.  ,  0.  , 76.  ,  2.  ,  1.  , 74.  ,  0.75,  0.  ,
       74.  ,  1.  ,  0.  , 72.  ,  6.  ,  0.  ])

In [ ]:
def get_all_stats_from_array(d_midi):
    _num_midi_features = 3
    _seq_length = int(len(d_midi) / _num_midi_features)
    stats = {}
    tones = []
    note_type = []
    rest = []
    for i in range(_seq_length):
        tones.append(d_midi[i*_num_midi_features+0])
    for i in range(_seq_length):
        note_type.append(d_midi[i*_num_midi_features+1])
    for i in range(_seq_length):
        rest.append(d_midi[i*_num_midi_features+2])
    if len(tones) == 0:
        print('This is an empty song.')
        return None
    stats['num_tones'] = len(tones)
    stats['tone_min'] = min(tones)
    stats['tone_max'] = max(tones)
    stats['tone_span'] = max(tones)-min(tones)
    stats['tones_unique'] = len(set(tones))
    rs = midi_statistics.repetitions(tones)
    for r in range(2, 10):
        if r in rs:
          stats['repetitions_{}'.format(r)] = rs[r]
        else:
          stats['repetitions_{}'.format(r)] = 0
    ml = midi_statistics.max_likelihood_scale(tones)
    stats['scale'] = ml[0]
    stats['scale_score'] = ml[1]
    stats['rest_max'] = max(rest)
    stats['average_rest'] = np.mean(rest)
    stats['num_null_rest'] = rest.count(0)
    stats['songlength'] = sum(rest) + sum(note_type)

    return stats

In [ ]:
def analyze_ground_truth(data):
  # for each data[i], data[i][:60] is the midi data. the rest is the lyrics data
  model_stats = {}
  model_stats['stats_scale_tot'] = 0
  model_stats['stats_repetitions_2_tot'] = 0
  model_stats['stats_repetitions_3_tot'] = 0
  model_stats['stats_span_tot'] = 0
  model_stats['stats_unique_tones_tot'] = 0
  model_stats['stats_avg_rest_tot'] = 0
  model_stats['num_of_null_rest_tot'] = 0
  model_stats['best_scale_score'] = 0
  model_stats['best_repetitions_2'] = 0
  model_stats['best_repetitions_3'] = 0
  model_stats['num_perfect_scale'] = 0
  model_stats['num_good_songs'] = 0
  model_stats['avg_songlength'] = 0

  for d in data:
    d_midi = d[:60]
    stats = get_all_stats_from_array(d_midi)
    # get stats
    model_stats['stats_scale_tot'] += stats['scale_score']
    model_stats['stats_repetitions_2_tot'] += float(stats['repetitions_2'])
    model_stats['stats_repetitions_3_tot'] += float(stats['repetitions_3'])
    model_stats['stats_unique_tones_tot'] += float(stats['tones_unique'])
    model_stats['stats_span_tot'] += stats['tone_span']
    model_stats['stats_avg_rest_tot'] += stats['average_rest']
    model_stats['num_of_null_rest_tot'] += stats['num_null_rest']
    model_stats['best_scale_score'] = max(stats['scale_score'], model_stats['best_scale_score'])
    model_stats['best_repetitions_2'] = max(stats['repetitions_2'], model_stats['best_repetitions_2'])
    model_stats['best_repetitions_3'] = max(stats['repetitions_3'], model_stats['best_repetitions_3'])
    model_stats['avg_songlength'] += stats['songlength']

    if stats['scale_score'] == 1.0 and stats['tones_unique'] > 3 \
        and stats['tone_span'] > 4 and stats['num_null_rest'] > 8 and stats['tone_span'] < 13\
        and stats['repetitions_2'] > 4:
        model_stats['num_good_songs'] += 1
    
  model_stats['avg_songlength'] /= len(data)

  return model_stats

In [ ]:
test_gt_stats = analyze_ground_truth(test_data)
valid_gt_stats = analyze_ground_truth(valid_data)
train_gt_stats = analyze_ground_truth(train_data)

In [ ]:
test_gt_stats, valid_gt_stats, train_gt_stats

({'stats_scale_tot': 1003.4500000000007,
  'stats_repetitions_2_tot': 7803.0,
  'stats_repetitions_3_tot': 4039.0,
  'stats_span_tot': 11326.0,
  'stats_unique_tones_tot': 6184.0,
  'stats_avg_rest_tot': 815.2499999999987,
  'num_of_null_rest_tot': 16380,
  'best_scale_score': 1.0,
  'best_repetitions_2': 18,
  'best_repetitions_3': 17,
  'num_perfect_scale': 0,
  'num_good_songs': 383,
  'avg_songlength': 43.25404376784015},
 {'stats_scale_tot': 1004.9500000000018,
  'stats_repetitions_2_tot': 7889.0,
  'stats_repetitions_3_tot': 4086.0,
  'stats_span_tot': 10946.0,
  'stats_unique_tones_tot': 6155.0,
  'stats_avg_rest_tot': 830.8999999999995,
  'num_of_null_rest_tot': 16142,
  'best_scale_score': 1.0,
  'best_repetitions_2': 18,
  'best_repetitions_3': 17,
  'num_perfect_scale': 0,
  'num_good_songs': 416,
  'avg_songlength': 43.13796384395813},
 {'stats_scale_tot': 10642.79999999999,
  'stats_repetitions_2_tot': 83330.0,
  'stats_repetitions_3_tot': 43417.0,
  'stats_span_tot': 1170

In [ ]:
model_path = 'saved_gan_models/saved_model_best_overall_mmd'

In [ ]:
test_gen_songs, test_model_stats = do_generate_songs(model_path, test_data)

In [ ]:
train_data = np.load('./data/processed_dataset_matrices/train_data_matrix.npy')
train_data.shape

(11149, 460)

In [ ]:
valid_data = np.load('./data/processed_dataset_matrices/valid_data_matrix.npy')
valid_data.shape

(1051, 460)

In [ ]:
valid_gen_songs, valid_model_stats = do_generate_songs(model_path, valid_data)

In [ ]:
train_gen_songs, train_model_stats = do_generate_songs(model_path, train_data)

In [ ]:
test_gen_songs = np.asarray(test_gen_songs)
test_gen_songs.shape

(1051, 20, 3)

In [ ]:
valid_get_songs = np.asarray(valid_gen_songs)
valid_get_songs.shape

(1051, 20, 3)

In [ ]:
train_get_songs = np.asarray(train_gen_songs)
train_get_songs.shape

(11149, 20, 3)

In [ ]:
test_model_stats

{'stats_scale_tot': 1051.0,
 'stats_repetitions_2_tot': 6694.0,
 'stats_repetitions_3_tot': 2630.0,
 'stats_span_tot': 11992.0,
 'stats_unique_tones_tot': 6805.0,
 'stats_avg_rest_tot': 338.50000000000006,
 'num_of_null_rest_tot': 17057,
 'best_scale_score': 1.0,
 'best_repetitions_2': 14,
 'best_repetitions_3': 12,
 'num_perfect_scale': 0,
 'num_good_songs': 582,
 'avg_songlength': 25.955280685061847}

In [ ]:
valid_model_stats

{'stats_scale_tot': 1051.0,
 'stats_repetitions_2_tot': 6616.0,
 'stats_repetitions_3_tot': 2619.0,
 'stats_span_tot': 12397.0,
 'stats_unique_tones_tot': 6941.0,
 'stats_avg_rest_tot': 347.1000000000005,
 'num_of_null_rest_tot': 16903,
 'best_scale_score': 1.0,
 'best_repetitions_2': 15,
 'best_repetitions_3': 13,
 'num_perfect_scale': 0,
 'num_good_songs': 551,
 'avg_songlength': 26.238582302568982}

In [ ]:
train_model_stats

{'stats_scale_tot': 11149.0,
 'stats_repetitions_2_tot': 70182.0,
 'stats_repetitions_3_tot': 27138.0,
 'stats_span_tot': 130290.0,
 'stats_unique_tones_tot': 73094.0,
 'stats_avg_rest_tot': 3476.5500000000447,
 'num_of_null_rest_tot': 182102,
 'best_scale_score': 1.0,
 'best_repetitions_2': 16,
 'best_repetitions_3': 14,
 'num_perfect_scale': 0,
 'num_good_songs': 6100,
 'avg_songlength': 25.76015786169163}

In [ ]:

generated_pitches = generated_songs[:, :, 0]
generated_lengths = generated_songs[:, :, 1]
generated_rests   = generated_songs[:, :, 2]

generated_pitches.shape, generated_lengths.shape, generated_rests.shape


((1051, 20), (1051, 20), (1051, 20))

In [ ]:

np.save(model_path + '/generated_songs',   generated_songs)
np.save(model_path + '/generated_pitches', generated_pitches)
np.save(model_path + '/generated_lengths', generated_lengths)
np.save(model_path + '/generated_rests',   generated_rests)
